# Yelp API - Lab



## Introduction 

Now that we've seen how the Yelp API works, it's time to put those API and Pandas skills to work in order to do some basic business analysis! Taking things a step further, you'll also independently explore how to perform pagination in order to retrieve a full results set from the Yelp API!

## Objectives

You will be able to:
* Create HTTP requests to get data from Yelp API
* Parse HTTP responses and save the information in a csv
* Perform pagination to retrieve troves of data!
* Write Pandas code to answer questions about your data 

## Problem Introduction

For this lab you will analyze the yelp data for a group of businesses to learn more about an industry. You will choose a type of business (Italian Restuarants, Nail Salons, Crossfit gyms) and a location to analyze. Then you will get data from the Yelp API, store that data in a SQL Database on AWS, and write queries to answer questions about the data. 


### Process:

1. Read through the data questions and the API documentation to determine which pieces of information you need to pull from the Yelp API.

2. Plan out what . One for the businesses and one for the reviews.

3. Create code to:
  - Perform a search of businesses using pagination
  - Parse the API response for specific data points
  - Save the data you pull as a csv

4. Use the functions above in a loop that will paginate over the results to retrieve all of the results. 

5. Create functions to:
  - Retrieve the reviews data of one business
  - Parse the reviews response for specific review data
  - Save the data you pull as a csv

6. Take all of the business IDs from your business search, and  using the 3 Python functions you've created, run your business IDs through a loop to get the reviews for each business and save them in a csv.

7. Write Pandas code to answer the following questions about your data.


Bonus Steps:  
- Place your helper functions in a package so that your final notebook only has the major steps listed.
- Rewrite your business search functions to be able take an argument for the type of business you are searching for.
- Add another group of businesses to your files.


 
## Data Questions:

- Which are the 5 most reviewed businesses?
- What is the highest rating recieved in your data set and how many businesses have that rating?
- What percentage of businesses have a rating greater than or  4.5?
- What percentage of businesses have a rating less than 3?
- What is the average rating of restaurants that have a price label of one dollar sign? Two dollar signs? Three dollar signs? 
- Return the text of the reviews for the most reviewed restaurant. 
- Return the name of the business with the most recent review. 
- Find the highest rated business and return text of the most recent review. If multiple business have the same rating, select the restaurant with the most reviews. 
- Find the lowest rated business and return text of the most recent review.  If multiple business have the same rating, select the restaurant with the least reviews. 


In [ ]:
#Movie Theaters, Brooklyn
#business info: biz_id,name,rating,review_count,price,
#reviews

## Part 1 - Understanding your data and question

Lok at the question and determine what data you will need to store in your database in order to answer the questions. Start to think about what tables will you want to create and what columns will you ahve for those tables. 

Look at the API documentation, and determine what fields of the API response you will match up with the columns you want in your Pandas Dataframes. 


https://www.yelp.com/developers/documentation/v3/get_started

## Part 2 - Create ETL pipeline for the business data from the API

Now that you know what data you need from the API, you want to write code that will execute a api call, parse those results and then insert the results into the DB.  

It is helpful to break this up into three different functions (*api call, parse results, and insert into DB*) and then you can write a function/script that pull the other three functions together. 

Let's first do this for the Business endpoint.

In [36]:
import requests
import json
import numpy as np
import pandas as pd

In [1]:
# Write a function to make a call to the yelp API

In [8]:
from keys import client_id,api_key
url =  'https://api.yelp.com/v3/businesses/search'
term='Movie Theaters'
location='Brooklyn'

url_params={
    'term': term.replace(' ','+'),
    'location': location.replace(' ','+'),
    'limit': 50
}



In [13]:
def yelp_call(url_params, api_key):
    # your code to make the yelp call
    headers = {
        'Authorization': 'Bearer {}'.format(api_key),
    }
    data= requests.get(url, headers=headers, params=url_params)
    return data

In [14]:
response = yelp_call(url_params, api_key)
response.status_code

200

In [47]:
response.json().keys()

dict_keys(['businesses', 'total', 'region'])

In [18]:
response.json()['total']

537

In [3]:
# write a function to parse the API response 
# so that you can easily insert the data in to the DB


In [48]:
data=response.json()['businesses']
data[0].keys()

dict_keys(['id', 'alias', 'name', 'image_url', 'is_closed', 'url', 'review_count', 'categories', 'rating', 'coordinates', 'transactions', 'location', 'phone', 'display_phone', 'distance'])

In [34]:
def parse_results(results):
    # your code to parse the result to make them easier to insert into the DB
    # create a container to hold our parsed data
    parsed_data = []
    businesses = results.json()['businesses']
    # loop through our business and parse each individual business
    for business in businesses:
        #print(business['name'])
        if 'price' in list(business.keys()):
            biz_list = [
                business['id'],
                business['name'], 
                business['review_count'], 
                business['rating'],
                business['price']
            ]
        else:
            biz_list = [
                business['id'],
                business['name'], 
                business['review_count'], 
                business['rating'],
                np.nan
            ]
    # in the loop, add each individual parsed  business to our data container
        parsed_data.append(biz_list)
    return parsed_data

In [38]:
parsed_data=parse_results(response)

In [49]:
parsed_data

[['M2denMcXzZPJm9RkYhStkQ',
  'Regal UA Sheepshead Bay IMAX & RPX',
  335,
  2.5,
  nan],
 ['c1onPljpeivCbdgYisVWNw', 'Nitehawk Cinema', 693, 4.5, nan],
 ['-m6RCdKiS5fC0wkZJHe2xg', 'Alpine Theatre', 282, 2.5, nan],
 ['H38pIONXN5yM4FMNtCMZqw', 'Nitehawk Cinema - Prospect Park', 167, 3.5, '$$'],
 ['_GH68RRjkwNbwT6LWIgm9Q', 'Williamsburg Cinemas', 334, 3.0, nan],
 ['-KrKo3-o8wAN5OQn1z_Tow', 'Cobble Hill Cinemas', 318, 3.5, nan],
 ['0AKsd2lA76JIvSc-1rns1g', 'Regal UA Court Street & RPX', 433, 2.0, nan],
 ['jCsNg4IjZz7AOFBkNR-jxw', 'Syndicated Bar Theater Kitchen', 207, 4.0, '$$'],
 ['hKP_L68yaJw6UOMMpIegDw', 'Cinemart Cinemas', 201, 4.0, nan],
 ['8hCw819wU7LkMG-tJHcpZA', 'AMC Fresh Meadows 7', 612, 4.0, nan],
 ['9bm1OLJZ_xkGUrOVQzNEig', 'BAM Rose Cinemas', 260, 4.0, '$$'],
 ['cDGEgWAA2RRm1H_27e4-dg', 'Kerasotes ShowPlace 14', 325, 4.5, nan],
 ['LKiCVg9QCXWJwZa1_Q63dw', 'Regal Battery Park', 512, 4.0, nan],
 ['vieBJCxpsKQauyTdkYQm5A', 'Regal UA Midway', 218, 3.0, nan],
 ['XjnUeIR7cD0RdB3BKE

In [1]:
# Write a function to take your parsed data and insert it into CSV

In [53]:
def data_save(parsed_results, csv_filename):
    # your code to save the current results with all of the other results. 
    # I would save the data every time you pull 50 results
    # in case something breaks in the process.
    cols=['ID','Name','Reviews','Rating','Price']
    df=pd.DataFrame(parsed_results,columns=cols)
    df.to_csv(csv_filename)

In [43]:
# Write a script that combines the three functions above into a single process.

# create a variable  to keep track of which result you are in. 
cur = 0
ready_data=[]
#set up a while loop to go through and grab the result 
while  cur < 537:
    #set the offset parameter to be where you currently are in the results 
    url_params['offset'] = cur
    #make your API call with the new offset number
    results = yelp_call(url_params, api_key)
    
    #after you get your results you can now use your function to parse those results
    parsed_results = parse_results(results)
    
    # use your function to insert your parsed results into the db
    #data_save(parsed_results, csv_file)
    ready_data.extend(parsed_results)
    #increment the counter by 50 to move on to the next results
    cur += 50

In [44]:
len(ready_data)

537

In [54]:
data_save(ready_data, 'yelp_data_biz.csv')

In [76]:
df=pd.read_csv('yelp_data_biz.csv',index_col=0)

df.head()

,ID,Name,Reviews,Rating,Price
0,M2denMcXzZPJm9RkYhStkQ,Regal UA Sheepshead Bay IMAX & RPX,335,2.5,NaN
1,c1onPljpeivCbdgYisVWNw,Nitehawk Cinema,693,4.5,NaN
2,-m6RCdKiS5fC0wkZJHe2xg,Alpine Theatre,282,2.5,NaN
3,H38pIONXN5yM4FMNtCMZqw,Nitehawk Cinema - Prospect Park,167,3.5,$$
4,_GH68RRjkwNbwT6LWIgm9Q,Williamsburg Cinemas,334,3.0,NaN


## Part 4 -  Create ETL pipeline for the restaurant review data from the API

You've done this for the Businesses, now you need to do this for reviews. You will follow the same process, but your functions will be specific to reviews.

In [58]:
# write Pandas code to pull back all of the business ids 
# you will need these ids to pull back the reviews for each restaurant
biz_ids=df['ID'].values

In [59]:
# write a function that takes a business id 
# and makes a call to the API for reivews
def yelp_review(biz_id, api_key):
    url =  'https://api.yelp.com/v3/businesses/{}/reviews'.format(biz_id)
    headers = {
        'Authorization': 'Bearer {}'.format(api_key),
    }
    data= requests.get(url, headers=headers)
    return data
    

In [61]:
review_data=yelp_review(biz_ids[0], api_key)

In [88]:
review_data.json()['reviews'][0]

{'id': 'Zub5hKsEOXQv2dvy6-XJCQ',
 'url': 'https://www.yelp.com/biz/regal-ua-sheepshead-bay-imax-and-rpx-brooklyn-2?adjust_creative=iiD4cWOR-Fz3fX4le1knTg&hrid=Zub5hKsEOXQv2dvy6-XJCQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=iiD4cWOR-Fz3fX4le1knTg',
 'text': "My childhood movie theater-- nicer than I remember it being lol. It's a movie theater, clean enough & gets the job done. On a Saturday night around 7 PM,...",
 'rating': 3,
 'time_created': '2020-03-03 11:05:21',
 'user': {'id': '96tc9KDtBj8V8zLWcBFluQ',
  'profile_url': 'https://www.yelp.com/user_details?userid=96tc9KDtBj8V8zLWcBFluQ',
  'image_url': 'https://s3-media3.fl.yelpcdn.com/photo/-cQz3qgwnOPQrLnBeMDqfg/o.jpg',
  'name': 'Christal Y.'}}

In [71]:
# Write a function to parse out the relevant information from the reviews
def parse_reviews(reviews, biz_id):
    parsed_data=[]
    data=reviews.json()['reviews']
    for i in data:
        review=[
            biz_id,
            i['text'],
            i['time_created']
        ]
        parsed_data.append(review)
    return parsed_data

In [72]:
parse_reviews(review_data, biz_ids[0])

[['M2denMcXzZPJm9RkYhStkQ',
  "My childhood movie theater-- nicer than I remember it being lol. It's a movie theater, clean enough & gets the job done. On a Saturday night around 7 PM,...",
  '2020-03-03 11:05:21'],
 ['M2denMcXzZPJm9RkYhStkQ',
  "My first time there and I thought it would've been an upgrade from the usual theatre we visit (since this is an imax... if that even matters lol). WRONG!!!...",
  '2020-02-18 18:59:15'],
 ['M2denMcXzZPJm9RkYhStkQ',
  "Listen to me TRUST ME , DO NOT I REPEAT DO NOT GO TO THIS PLACE. I'm currently here and boy I should have known better . It stinks badly , it's dirty af I...",
  '2020-01-15 02:29:24']]

In [68]:
# write a function to insert the parsed data into the reviews table
def data_save2(parsed_results, csv_filename):
    # your code to save the current results with all of the other results. 
    # I would save the data every time you pull 50 results
    # in case something breaks in the process.
    cols=['ID','Text','Time']
    df=pd.DataFrame(parsed_results,columns=cols)
    df.to_csv(csv_filename)

In [73]:
# combine the functions above into a single script  
all_reviews=[]
for biz_id in biz_ids:
    review=yelp_review(biz_id,api_key)
    
    parsed_reviews=parse_reviews(review,biz_id)
    
    all_reviews.extend(parsed_reviews)
    
len(all_reviews)

1573

In [74]:
data_save2(all_reviews,'yelp_reviews')

In [85]:
df2=pd.read_csv('yelp_reviews',index_col=0)
df2.head()

,ID,Text,Time
0,M2denMcXzZPJm9RkYhStkQ,My childhood movie theater-- nicer than I reme...,2020-03-03 11:05:21
1,M2denMcXzZPJm9RkYhStkQ,My first time there and I thought it would've ...,2020-02-18 18:59:15
2,M2denMcXzZPJm9RkYhStkQ,"Listen to me TRUST ME , DO NOT I REPEAT DO NOT...",2020-01-15 02:29:24
3,c1onPljpeivCbdgYisVWNw,Love coming here. They usually screen all the ...,2020-02-03 11:23:06
4,c1onPljpeivCbdgYisVWNw,Came here for the first time tonight and loved...,2020-01-31 22:03:15


## Part 5 -  Write Pandas code that will answer the questions posed. 

Now that your data is saved in CSVs, you can answer the questions. 

 
## Data Questions:

- Which are the 5 most reviewed businesses?
- What is the highest rating recieved in your data set and how many businesses have that rating?
- What percentage of businesses have a rating greater than or  4.5?
- What percentage of businesses have a rating less than 3?
- What is the average rating of restaurants that have a price label of one dollar sign? Two dollar signs? Three dollar signs? 
- Return the text of the reviews for the most reviewed restaurant. 
- Return the name of the business with the most recent review. 
- Find the highest rated business and return text of the most recent review. If multiple business have the same rating, select the restaurant with the most reviews. 
- Find the lowest rated business and return text of the most recent review.  If multiple business have the same rating, select the restaurant with the least reviews. 


In [92]:
# 5 most reviewed
top5_reviewed=df.sort_values(by='Reviews',ascending=False).head(5)
top5_reviewed['Name']

493        The Metropolitan Museum of Art
460    American Museum of Natural History
383                 Empire State Building
319              The Museum of Modern Art
478                       Top of the Rock
Name: Name, dtype: object

In [95]:
# Highest Rating + how many Businesses have it
highest_ratings=df.sort_values(by='Rating', ascending=False)
highest_ratings.groupby('Rating').size()

Rating
1.5      4
2.0     16
2.5     34
3.0     52
3.5    108
4.0    172
4.5    127
5.0     24
dtype: int64

In [114]:
# % with rating>=4.5
rating_percents=df[df['Rating']>=4.5]['Rating'].count()/df['Rating'].count()
rating_percents*100

28.11918063314711

In [115]:
# % with rating<3
rating_percents=df[df['Rating']<3]['Rating'].count()/df['Rating'].count()
rating_percents*100

10.05586592178771

In [120]:
#Average rating of restaurant based on Price
df.groupby('Price')['Rating'].mean()

Price
$       3.916667
$$      3.771429
$$$     4.000000
$$$$    4.000000
Name: Rating, dtype: float64

In [130]:
#Return the text of the reviews for the most reviewed restaurant.
df_all=pd.merge(df,df2,on='ID')
most_reviews=df_all['Reviews'].max()
df_all[df_all['Reviews']==most_reviews]['Text'].values

array(["Celebrating 150 years and still as strong and exciting as ever. Second time here and still haven't see everything. There is so much to see that a ticket is...",
       'One of the best museums I went to in NY. \n\nI went especially to see the fashion exhibition, Heavenly Bodies: Fashion and The Catholic Imagination. Great...',
       "Come to the MET on several occasions through and like always you can never finish more than 4 exhibits and that's rushing it.  \n\nThey always have a seasonal..."],
      dtype=object)

In [133]:
#Return the name of the business with the most recent review.
df_all[df_all['Time']==df_all['Time'].max()]['Name']

1389    Color Factory
Name: Name, dtype: object

In [144]:
#Find the highest rated business and return text of the most recent review. 
#If multiple business have the same rating, select the restaurant with the most reviews.
highest_rated=df_all[df_all['Rating']==df_all['Rating'].max()]
high_most_reviews=highest_rated[highest_rated['Reviews']==highest_rated['Reviews'].max()]
high_most_reviews.sort_values(by='Time',ascending=False).head(1)[['Name','Text']]

,Name,Text
1005,Blogologues,So funny and so relevant! I saw Blogologues wh...


In [145]:
#Find the lowest rated business and return text of the most recent review. 
#If multiple business have the same rating, select the restaurant with the least reviews.
lowest_rated=df_all[df_all['Rating']==df_all['Rating'].min()]
low_least_reviews=lowest_rated[lowest_rated['Reviews']==lowest_rated['Reviews'].min()]
low_least_reviews.sort_values(by='Time',ascending=False).head(1)[['Name','Text']]

,Name,Text
412,Columbia Park 12,The night I went there to see Godzilla King of...


# Extra Reference help

###  Pagination

Returning to the Yelp API, the [documentation](https://www.yelp.com/developers/documentation/v3/business_search) also provides us details regarding the API limits. These often include details about the number of requests a user is allowed to make within a specified time limit and the maximum number of results to be returned. In this case, we are told that any request has a maximum of 50 results per request and defaults to 20. Furthermore, any search will be limited to a total of 1000 results. To retrieve all 1000 of these results, we would have to page through the results piece by piece, retriving 50 at a time. Processes such as these are often refered to as pagination.

Now that you have an initial response, you can examine the contents of the json container. For example, you might start with ```response.json().keys()```. Here, you'll see a key for `'total'`, which tells you the full number of matching results given your query parameters. Write a loop (or ideally a function) which then makes successive API calls using the offset parameter to retrieve all of the results (or 5000 for a particularly large result set) for the original query. As you do this, be mindful of how you store the data. 

**Note: be mindful of the API rate limits. You can only make 5000 requests per day, and APIs can make requests too fast. Start prototyping small before running a loop that could be faulty. You can also use time.sleep(n) to add delays. For more details see https://www.yelp.com/developers/documentation/v3/rate_limiting.**

***Below is sample code that you can use to help you deal with the pagination parameter and bring all of the functions together.***


***Also, something might cause your code to break while it is running. You don't want to constantly repull the same data when this happens, so you should insert the data into the database as you call and parse it, not after you have all of the data***


In [ ]:
# create a variable  to keep track of which result you are in. 
cur = 0

#set up a while loop to go through and grab the result 
while cur < num and cur < 1000:
    #set the offset parameter to be where you currently are in the results 
    url_params['offset'] = cur
    #make your API call with the new offset number
    results = yelp_call(url_params, api_key)
    
    #after you get your results you can now use your function to parse those results
    parsed_results = parse_results(results)
    
    # use your function to insert your parsed results into the db
    data_save(parsed_results)
    #increment the counter by 50 to move on to the next results
    cur += 50